<a href="https://colab.research.google.com/github/Himaja2304/AL-CASE-STUDY/blob/main/Bidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from collections import deque

class SherlockMystery:
    def __init__(self, clue_graph, mystery_words):
        self.clue_graph = clue_graph  # Graph representation of clues
        self.mystery_words = mystery_words  # Hangman words for clues

    def play_hangman(self, word, max_attempts=6):
        """Simulates a Hangman game where Sherlock must solve a word puzzle"""
        word = word.lower()
        guessed_letters = set()
        attempts = 0
        hidden_word = ["_" if char.isalpha() else char for char in word]

        while "_" in hidden_word and attempts < max_attempts:
            print("\n🔎 Current word:", " ".join(hidden_word))
            guess = random.choice("abcdefghijklmnopqrstuvwxyz")  # AI guesses randomly
            print(f"🕵️ Sherlock guesses: {guess}")

            if guess in word:
                for i, char in enumerate(word):
                    if char == guess:
                        hidden_word[i] = guess
            else:
                attempts += 1
                print(f"❌ Wrong guess! Attempts left: {max_attempts - attempts}")

        return "".join(hidden_word) == word  # True if solved

    def bidirectional_search(self, start_clue, target_clue):
        """Performs Bidirectional Search with Hangman clue-solving"""
        forward_queue = deque([start_clue])
        backward_queue = deque([target_clue])
        forward_visited = {start_clue: None}
        backward_visited = {target_clue: None}

        while forward_queue and backward_queue:
            # Expand forward search
            if self.expand_search(forward_queue, forward_visited, backward_visited):
                return self.build_path(forward_visited, backward_visited)

            # Expand backward search
            if self.expand_search(backward_queue, backward_visited, forward_visited):
                return self.build_path(forward_visited, backward_visited)

        return None  # No connection found

    def expand_search(self, queue, visited, opposite_visited):
        """Expands search in one direction"""
        current_clue = queue.popleft()

        # If this clue requires solving Hangman, do it before proceeding
        if current_clue in self.mystery_words:
            print(f"\n📝 Clue found: {current_clue}. Sherlock must solve Hangman puzzle!")
            if not self.play_hangman(self.mystery_words[current_clue]):
                print(f"🚧 Sherlock failed to solve: {current_clue}. Investigation stalled!")
                return False  # Stop this path if Hangman fails

        # Expand to neighboring clues
        for next_clue in self.clue_graph.get(current_clue, []):
            if next_clue not in visited:
                visited[next_clue] = current_clue
                queue.append(next_clue)

                # If paths meet, return True
                if next_clue in opposite_visited:
                    return True

        return False

    def build_path(self, forward_visited, backward_visited):
        """Builds the final path by merging forward and backward search results"""
        meeting_point = next(clue for clue in forward_visited if clue in backward_visited)

        # Construct forward path
        path = []
        clue = meeting_point
        while clue:
            path.append(clue)
            clue = forward_visited[clue]
        path.reverse()

        # Construct backward path
        clue = backward_visited[meeting_point]
        while clue:
            path.append(clue)
            clue = backward_visited[clue]

        return path

# Example Graph: Connecting clues from the crime scene to the culprit
clue_connections = {
    "Crime Scene": ["Bloody Knife", "Footprint", "Witness"],
    "Bloody Knife": ["DNA Sample", "Fingerprint"],
    "Footprint": ["Shoe Brand"],
    "Witness": ["Statement", "Suspect Sketch"],
    "DNA Sample": ["Suspect"],
    "Fingerprint": ["Suspect"],
    "Shoe Brand": ["Store Records"],
    "Statement": ["Alibi Check"],
    "Suspect": ["Culprit"],
}

# Mystery Words (Hangman puzzles for some clues)
mystery_words = {
    "Bloody Knife": "dagger",
    "Fingerprint": "identity",
    "Shoe Brand": "nike",
    "Statement": "witness",
    "Suspect": "moriarty",
}

# Sherlock starts at "Crime Scene" and tries to connect to "Culprit"
sherlock_ai = SherlockMystery(clue_connections, mystery_words)
solution_path = sherlock_ai.bidirectional_search("Crime Scene", "Culprit")

# Output Results
if solution_path:
    print("\n🕵️ Sherlock solved the case! Clue path:", " → ".join(solution_path))
else:
    print("\n❌ No solution found.")



❌ No solution found.
